# Experimental data preprocessing - second stage

Integrate the recall and recognition data sets with the predictions of the three models (Bayesian language model, word associations form cooccurrrences model, word associations from word association norms model).

This process is quick. Total running time is around 30 seconds.

In [1]:
from __future__ import division

# Standard library imports
import os
import cPickle as pickle

# Third party imports
import pandas
import numpy
import configobj

# Local imports
from utils import datautils
from utils import topicmodels
from utils import utils

In [2]:
cache_directory = '../cache'
cache_fullpath = lambda path: os.path.join(cache_directory, path)

## Load up vocab

In [3]:
vocabulary = open(cache_fullpath('bnc_vocab_49328.txt')).read().split()
vocab = datautils.Vocab(vocabulary)

## Load up recall data

In [4]:
Df = {}
Df['recall'] = pandas.read_pickle(cache_fullpath('brisbane_06b643a_recall_results.pkl'))

In [5]:
Df['recall'].head()

,session,subject,age,sex,slide,completed,text,readingtime,word,accuracy,response
0,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,apparently,True,11-apparently
1,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,there,True,11-there
2,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,is,True,11-is
3,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,no,True,11-no
4,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,case,True,11-case


Filter data frame by only responses where the recalled word is in the vocabulary.

In [6]:
I = map(lambda word: True if word in vocabulary else False, Df['recall']['word'])
Df['recall'] = Df['recall'][I]

In [7]:
Df['recall'].head()

,session,subject,age,sex,slide,completed,text,readingtime,word,accuracy,response
0,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,apparently,True,11-apparently
5,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,law,True,11-law
9,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,directive,True,11-directive
11,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,faith,True,11-faith
12,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,family,True,11-family


## Load up recognition data

In [8]:
Df['recognition'] = pandas.read_pickle(cache_fullpath('brisbane_06b643a_recognition_results.pkl'))

Df['recognition'].head()

,session,subject,age,sex,slide,completed,text,readingtime,word,expected,order,hit,response,correct,rt,stimulus
0,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,purple,True,0,True,True,True,1.002,45-purple
1,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,tastefully,False,1,True,False,True,0.917,45-tastefully
2,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,cataract,True,2,True,True,True,1.199,45-cataract
3,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,sack,True,3,True,True,True,0.710,45-sack
4,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,relic,False,4,True,False,True,1.040,45-relic


## Load up cooccurrence probabilities


In [9]:
with open(cache_fullpath('word_associates_from_cooccurrence_statistics.pkl')) as f:
    cooccurrence_predictions = pickle.load(f)

Confirm that for every recalled word or recognition stimulus, we have a predicted probability. We ignore here the word 'dhow', which was one of the recognition memory test items for text 23. I'm not sure how that got in given that we aimed to restrict the test items to word in a standard vocabulary test.

In [10]:
def check_predictions_items(Df, predictions):
    
    '''Confirm that for every recalled word or recognition stimulus, 
    we have a predicted probability. We ignore here the word 'dhow', 
    which was one of the recognition memory test items for text 23. 
    I'm not sure how that got in given that we aimed to restrict 
    the test items to word in a standard vocabulary test.'''
    
    words = []
    texts = []
    for key in predictions.keys():
        text_id, word = key.split('-')
        words.append(word)
        texts.append(int(text_id))

    assert all(map(lambda recalled_word: recalled_word in words, Df['recall']['word']))
    
    for word in Df['recognition']['word']:
        try:
            assert word in words
        except AssertionError:
            assert word == u'dhow', word
            
    return True
            
assert check_predictions_items(Df, cooccurrence_predictions)

Add the cooccurrence predictions to the recall data frame.

In [11]:
Df['recall']['cooccurrence.predictions'] = map(cooccurrence_predictions.get, 
                                               Df['recall']['response'])

Df['recall'].head()

,session,subject,age,sex,slide,completed,text,readingtime,word,accuracy,response,cooccurrence.predictions
0,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,apparently,True,11-apparently,0.000239
5,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,law,True,11-law,0.002576
9,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,directive,True,11-directive,0.001044
11,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,faith,True,11-faith,0.000401
12,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,family,True,11-family,0.001195


Do the same for the recognition results.

In [12]:
Df['recognition']['cooccurrence.predictions'] = map(cooccurrence_predictions.get, 
                                                    Df['recognition']['stimulus'])

Df['recognition'].head()

,session,subject,age,sex,slide,completed,text,readingtime,word,expected,order,hit,response,correct,rt,stimulus,cooccurrence.predictions
0,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,purple,True,0,True,True,True,1.002,45-purple,0.000101
1,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,tastefully,False,1,True,False,True,0.917,45-tastefully,0.000010
2,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,cataract,True,2,True,True,True,1.199,45-cataract,0.000021
3,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,sack,True,3,True,True,True,0.710,45-sack,0.000310
4,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,relic,False,4,True,False,True,1.040,45-relic,0.000025


## Load the model's posterior predictions

In [13]:
with open(cache_fullpath('posterior_predictions.1001.pkl'),'rb') as f:
    posterior_predictions = pickle.load(f)

Now, we have to extract the posterior probability of each 'text-word' stimulus item from the `posterior_predictions` matrix. Remember, the `posterior_predictions` matrix is $J \times V$, where $J$ and $V$ are the number of texts and words in the vocabulary, respectively. We'll then check that we have posterior predictions for all items in the behavioural data.

In [14]:
corpus_data = utils.loadnpz('bnc_78723408_250_500_49328.npz.bz2', 
                            cache=cache_directory,
                            verbose=False)

word2index = {w:i for i,w in enumerate(corpus_data['vocabulary'])}
index2word = {i:w for i,w in enumerate(corpus_data['vocabulary'])}

posterior_predictions_items = {}
for item in list(Df['recall']['response'].unique())\
            + list(Df['recognition']['stimulus'].unique()):
    
    try:
        
        n, word = item.split('-')
        w_i = word2index[word]
        text_id = 'text_%d' % (int(n)-1)
        posterior_predictions_items[item] = posterior_predictions[text_id][w_i]
        
    except KeyError:
        assert item == '23-dhow', item

assert check_predictions_items(Df, posterior_predictions_items)

Add the posterior predictions to the recall memory data set.

In [15]:
Df['recall']['posterior.predictions'] = map(posterior_predictions_items.get, Df['recall']['response'])

Df['recall'].head()

,session,subject,age,sex,slide,completed,text,readingtime,word,accuracy,response,cooccurrence.predictions,posterior.predictions
0,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,apparently,True,11-apparently,0.000239,0.000114
5,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,law,True,11-law,0.002576,0.009909
9,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,directive,True,11-directive,0.001044,0.004320
11,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,faith,True,11-faith,0.000401,0.000932
12,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,family,True,11-family,0.001195,0.001698


And do the same for the recognition memory data.

In [16]:
Df['recognition']['posterior.predictions'] = map(posterior_predictions_items.get, 
                                                    Df['recognition']['stimulus'])

Df['recognition'].head()

,session,subject,age,sex,slide,completed,text,readingtime,word,expected,order,hit,response,correct,rt,stimulus,cooccurrence.predictions,posterior.predictions
0,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,purple,True,0,True,True,True,1.002,45-purple,0.000101,0.000303
1,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,tastefully,False,1,True,False,True,0.917,45-tastefully,0.000010,0.000001
2,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,cataract,True,2,True,True,True,1.199,45-cataract,0.000021,0.000040
3,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,sack,True,3,True,True,True,0.710,45-sack,0.000310,0.000223
4,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,relic,False,4,True,False,True,1.040,45-relic,0.000025,0.000021


## Load up the associations data

In [17]:
with open(cache_fullpath('word_associates_from_association_norms.pkl'), 'rb') as f:
    associations = pickle.load(f)

In [18]:
assert check_predictions_items(Df, associations)

Add the association predictions to the recall and recognition data frames.

In [19]:
Df['recall']['association.predictions'] = map(associations.get, 
                                              Df['recall']['response'])

Df['recall'].head()

,session,subject,age,sex,slide,completed,text,readingtime,word,accuracy,response,cooccurrence.predictions,posterior.predictions,association.predictions
0,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,apparently,True,11-apparently,0.000239,0.000114,0.000003
5,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,law,True,11-law,0.002576,0.009909,0.003213
9,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,directive,True,11-directive,0.001044,0.004320,0.000004
11,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,faith,True,11-faith,0.000401,0.000932,0.000123
12,186a069,4ba33f7,29,Male,96a7502,True,11,60.165,family,True,11-family,0.001195,0.001698,0.001264


In [20]:
Df['recognition']['association.predictions'] = map(associations.get, 
                                                   Df['recognition']['stimulus'])

Df['recognition'].head()

,session,subject,age,sex,slide,completed,text,readingtime,word,expected,order,hit,response,correct,rt,stimulus,cooccurrence.predictions,posterior.predictions,association.predictions
0,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,purple,True,0,True,True,True,1.002,45-purple,0.000101,0.000303,1.087610e-03
1,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,tastefully,False,1,True,False,True,0.917,45-tastefully,0.000010,0.000001,4.035587e-07
2,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,cataract,True,2,True,True,True,1.199,45-cataract,0.000021,0.000040,2.421352e-06
3,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,sack,True,3,True,True,True,0.710,45-sack,0.000310,0.000223,2.474474e-04
4,186a069,4ba33f7,29,Male,d69884d,True,45,62.805,relic,False,4,True,False,True,1.040,45-relic,0.000025,0.000021,6.658718e-06


## Write new data sets 

In [21]:
recall_results_filename = 'experiment_brisbane_recall_memory_tests_results.csv'
Df['recall'].to_csv(cache_fullpath(recall_results_filename))
assert utils.checksum(cache_fullpath(recall_results_filename))\
  == '0813474b891492101f83d815308b03a85627e9243623a0876731a85047fe397d'

recognition_results_filename = 'experiment_brisbane_recognition_memory_tests.csv'
Df['recognition'].to_csv(cache_fullpath(recognition_results_filename))
assert utils.checksum(cache_fullpath(recognition_results_filename))\
  == 'b4abc880d87db07068875022278d04bce782397738b85da0101f1253002b803d'